https://habr.com/ru/articles/566568/

In [ ]:
# устанавливаем необходимые для работы в colab зависимости
# !pip install plotly chart_studio

In [46]:
import os
import re
import requests
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
# import chart_studio.plotly as py
import plotly.graph_objects as go
import json

## Пример 1

У нас есть следующий датафрейь

In [92]:
df = pd.DataFrame({
    'Vendor Name': ['AKJ Education', 'AKJ Education', 'Amazon', 'Amazon', 'Amazon', 'Flipkart'],
    'Category': ['Books', 'Computers & Tablets', 'Books', 'Computers & Tablets', 'Other', 'Books'],
    'Count': [84688, 10045, 12944, 42165, 4150, 10230]
})
df

,Vendor Name,Category,Count
0,AKJ Education,Books,84688
1,AKJ Education,Computers & Tablets,10045
2,Amazon,Books,12944
3,Amazon,Computers & Tablets,42165
4,Amazon,Other,4150
5,Flipkart,Books,10230


Мы хотим построить Sankey диаграмму где источниками будет `Vendor Name`, а целями `Category`

Сначла создаем узлы  
Для этого мы берем все уникаьные занчения из испточников и целей выстраиваем в ряд и даем им индексы  


In [107]:
node_colors = ["rgba(42, 157, 143, 1)", "rgba(38, 70, 83, 1)", "rgba(233, 196, 106, 1)", "rgba(244, 162, 97, 1)", "rgba(231, 111, 81, 1)", "rgba(230, 57, 70, 1)"]
nodes_with_indexes = {key: [val, node_colors[val]] for val, key in enumerate(df['Vendor Name'].unique().tolist() + df['Category'].unique().tolist())}
nodes_with_indexes

{'AKJ Education': [0, 'rgba(42, 157, 143, 1)'],
 'Amazon': [1, 'rgba(38, 70, 83, 1)'],
 'Flipkart': [2, 'rgba(233, 196, 106, 1)'],
 'Books': [3, 'rgba(244, 162, 97, 1)'],
 'Computers & Tablets': [4, 'rgba(231, 111, 81, 1)'],
 'Other': [5, 'rgba(230, 57, 70, 1)']}

Создадим цвета связей    
цвет связи должен быть как цвет источника, из которого она идет  
Тут не берем уникальные источники, так как у нас связей будет столько сколько значений в столбце источников  

In [108]:
link_color = [nodes_with_indexes[source][1].replace(', 1)', ', 0.2)') for source in df['Vendor Name']]
link_color

['rgba(42, 157, 143, 0.2)',
 'rgba(42, 157, 143, 0.2)',
 'rgba(38, 70, 83, 0.2)',
 'rgba(38, 70, 83, 0.2)',
 'rgba(38, 70, 83, 0.2)',
 'rgba(233, 196, 106, 0.2)']

Создадим источники

In [94]:
df['source'] = df['Vendor Name'].apply(lambda x: nodes_with_indexes[x][0])

Создадим цели

In [96]:
df['target'] = df['Category'].apply(lambda x: nodes_with_indexes[x][0])

Посчитаем суммарное значение value для каждого источника, чтобы в ховерах показать процент от общего  

In [97]:
df['sum_value'] = df.groupby('Vendor Name')['Count'].transform('sum')

In [98]:
df['value_percent'] = round(df['Count'] * 100 / df['sum_value'], 2)
df['value_percent'] = df['value_percent'].apply(lambda x: f"{x}%")
df

,Vendor Name,Category,Count,source,target,sum_value,value_percent
0,AKJ Education,Books,84688,0,3,94733,89.4%
1,AKJ Education,Computers & Tablets,10045,0,4,94733,10.6%
2,Amazon,Books,12944,1,3,59259,21.84%
3,Amazon,Computers & Tablets,42165,1,4,59259,71.15%
4,Amazon,Other,4150,1,5,59259,7.0%
5,Flipkart,Books,10230,2,3,10230,100.0%


In [109]:
# Sankey plot setup
fig = go.Figure(data=[go.Sankey(
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 10,
      thickness = 15,
      line = dict(color = "black", width = 0.1),
      label =  list(nodes_with_indexes.keys()),
      color = node_colors
    ),
    link = dict(
      source = df['source'],
      target = df['target'],
      value  = df['Count'],
      label = df['value_percent'],
      color = link_color
  )
)])

layout = dict(
        title = "Draw Sankey Diagram",
        height = 772,
        font = dict(
        size = 10),)

fig.update_layout(layout)  
  

## Пример 2

### Подготовка данных

In [110]:
PATH_TO_CSV = 'https://raw.githubusercontent.com/rusantsovsv/senkey_tutorial/main/csv/senkey_data_tutorial.csv'

In [111]:
df = pd.read_csv(PATH_TO_CSV)
df.head()

,user_id,event_timestamp,event_name
0,0003da5682126e66414958d58022fea7,2021-05-21 18:31:09.677329001,app_opened_via_icon
1,0003da5682126e66414958d58022fea7,2021-05-21 18:31:09.679674004,history_opened
2,0003da5682126e66414958d58022fea7,2021-05-21 18:31:09.679818007,sales_category_selected
3,0003da5682126e66414958d58022fea7,2021-05-21 18:31:09.688039010,app_remove
4,00058a2e86c1e535e63b980fbda964b3,2021-05-21 18:30:29.121452000,app_remove


In [112]:
df.sort_values(['user_id', 'event_timestamp'], inplace=True)

добавим шаги событий

In [113]:
df['step'] = df.groupby('user_id').cumcount()+1

добавим узлы источники (это сами события)

In [114]:
df['source'] = df.event_name

добавим целевые узлы

In [115]:
df['target'] = df.groupby('user_id')['source'].shift(-1)

удалим лишнюю колонку `event_name`

In [116]:
df.drop('event_name', axis=1, inplace=True)
df.head()

,user_id,event_timestamp,step,source,target
0,0003da5682126e66414958d58022fea7,2021-05-21 18:31:09.677329001,1,app_opened_via_icon,history_opened
1,0003da5682126e66414958d58022fea7,2021-05-21 18:31:09.679674004,2,history_opened,sales_category_selected
2,0003da5682126e66414958d58022fea7,2021-05-21 18:31:09.679818007,3,sales_category_selected,app_remove
3,0003da5682126e66414958d58022fea7,2021-05-21 18:31:09.688039010,4,app_remove,NaN
4,00058a2e86c1e535e63b980fbda964b3,2021-05-21 18:30:29.121452000,1,app_remove,NaN


In [138]:
# удалим все пары source-target, шаг которых превышает 7
# и сохраним полученную таблицу в отдельную переменную
df_comp = df[df['step'] <= 7].reset_index(drop=True)

### Создание индексов для source

Важным следующим шагом в подготовке данных является создание индексов для source.   
На каждом следующем шаге target становится source,   
и чтобы диаграмма коррректно генерировалась нужна правильная индексация source на каждом шаге.

у каждого узла индексы испточников не должоны начинаться с нуля,  
поэтому нужно проиндексировать все элементы узлов разными индексами

In [139]:
source_indexes = {}
count = 0

# получаем индексы источников
for step in df_comp.step.unique():
    source_indexes[step] = {}
    source_indexes[step]['sources'] = df_comp[df_comp.step == step]['source'].unique().tolist()
    source_indexes[step]['sources_index'] = []
    for _ in source_indexes[step]['sources']:
        source_indexes[step]['sources_index'].append(count)
        count += 1
# для каждого шага соединим списки в словарь, где ключом будет источник, а значение индекс        
for step in source_indexes:
    source_indexes[step]['sources_dict'] = {}
    for source, indx in zip(source_indexes[step]['sources'], source_indexes[step]['sources_index']):
        source_indexes[step]['sources_dict'][source] = indx           

In [119]:
source_indexes[1]['sources'][:5]

['app_opened_via_icon',
 'app_remove',
 'item_opened',
 'app_opened_from_market',
 'first_open']

In [120]:
source_indexes[1]['sources_index'][:5]

[0, 1, 2, 3, 4]

In [121]:
list(source_indexes[1]['sources_dict'].items())[:5]

[('app_opened_via_icon', 0),
 ('app_remove', 1),
 ('item_opened', 2),
 ('app_opened_from_market', 3),
 ('first_open', 4)]

### Генерация цветов для source

Для более наглядного представления можно разукрасить каждый source-target в разные цвета. Я рассмотрел 2 способа - случайная генерация и ручной выбор цветов.

Цвета выберем в цветовой модели RGBA. Это необходимо, чтобы сделать каналы source-target более прозрачными, по отношению к блокам для лучшей читаемости схемы.

Цвет будем генерировать для каждого уникального источника.

In [122]:
def colors_for_sources(df_comp, mode):
    """Генерация цветов rgba

    Args:
        mode (str): сгенерировать случайные цвета, если 'random', а если 'custom' - 
                    использовать заранее подготовленные
    Returns:
        dict: словарь с цветами, соответствующими каждому индексу
    """    
    colors_dict = {}
    
    if mode == 'random':
        for source in df_comp.source.unique():
            r, g, b = np.random.randint(255, size = 3)
            colors_dict[source] = f'rgba({r}, {g}, {b}, 1)'
    elif mode == 'custom':
        # colors = requests.get('https://raw.githubusercontent.com/rusantsovsv/senkey_tutorial/main/json/colors_senkey.json').json()
        colors = {"custom_colors": ["rgba(42, 157, 143, 1)", "rgba(38, 70, 83, 1)", "rgba(233, 196, 106, 1)", "rgba(244, 162, 97, 1)", "rgba(231, 111, 81, 1)", "rgba(230, 57, 70, 1)", "rgba(168, 218, 220, 1)", "rgba(69, 123, 157, 1)", "rgba(29, 53, 87, 1)", "rgba(107, 112, 92, 1)", "rgba(183, 183, 164, 1)", "rgba(221, 190, 169, 1)", "rgba(142, 202, 230, 1)", "rgba(33, 158, 188, 1)", "rgba(2, 48, 71, 1)", "rgba(255, 183, 3, 1)", "rgba(251, 133, 0, 1)", "rgba(153, 217, 140, 1)", "rgba(118, 200, 147, 1)", "rgba(52, 160, 164, 1)", "rgba(26, 117, 159, 1)", "rgba(247, 37, 133, 1)", "rgba(181, 23, 158, 1)", "rgba(114, 9, 183, 1)", "rgba(86, 11, 173, 1)", "rgba(72, 149, 239, 1)", "rgba(76, 201, 240, 1)", "rgba(242, 132, 130, 1)", "rgba(132, 165, 157, 1)", "rgba(246, 189, 96, 1)", "rgba(187, 62, 3, 1)", "rgba(204, 213, 174, 1)", "rgba(233, 237, 201, 1)", "rgba(254, 250, 224, 1)", "rgba(212, 163, 115, 1)", "rgba(96, 108, 56, 1)", "rgba(40, 54, 24, 1)", "rgba(249, 199, 79, 1)", "rgba(144, 190, 109, 1)", "rgba(67, 170, 139, 1)", "rgba(77, 144, 142, 1)", "rgba(87, 117, 144, 1)", "rgba(39, 125, 161, 1)", "rgba(239, 71, 111, 1)", "rgba(255, 209, 102, 1)", "rgba(6, 214, 160, 1)", "rgba(17, 138, 178, 1)", "rgba(7, 59, 76, 1)", "rgba(0, 53, 102, 1)", "rgba(226, 175, 255, 1)", "rgba(248, 173, 157, 1)", "rgba(211, 211, 211, 1)", "rgba(254, 228, 64, 1)", "rgba(241, 91, 181, 1)", "rgba(155, 93, 229, 1)", "rgba(0, 187, 249, 1)", "rgba(0, 245, 212, 1)", "rgba(130, 192, 204, 1)", "rgba(128, 185, 24, 1)", "rgba(43, 147, 72, 1)"]}
        for i, source in enumerate(df_comp.source.unique()):
            colors_dict[source] = colors["custom_colors"][i]
    return colors_dict            

In [ ]:
# генерим цвета
colors_dict = colors_for_sources(df_comp, mode='custom')
colors_dict

### Создаем словарь с данными

Диаграмму будем отрисовывать с помощью Plotly. Для корректной (и более полной) отрисовки нужны следующие данные:

sources - список с индексами source;

targets - список с индексами target;

values - количество уникальных пользователей, совершивших переход между узлами source-target ("объем" потока между узлами);

labels - названия узлов;

colors_labels - цвет узлов;

link_color - цвет потоков между узлами;

link_text - дополнительная информация.

Также не будем брать переходы, с малым количеством пользователей  
для этого используемтся параметр `frac`

In [124]:
def percent_users(sources, targets, values):
    
    """
    Расчет уникальных id в процентах (для вывода в hover text каждого узла)
    
    Args:
        sources (list): список с индексами source.
        targets (list): список с индексами target.
        values (list): список с "объемами" потоков.
        
    Returns:
        list: список с "объемами" потоков в процентах
    """
    
    # объединим источники и метки и найдем пары
    zip_lists = list(zip(sources, targets, values))
    
    new_list = []
    
    # подготовим список словарь с общим объемом трафика в узлах
    unique_dict = {}
    
    # проходим по каждому узлу
    for source, target, value in zip_lists:
        if source not in unique_dict:
            # находим все источники и считаем общий трафик
            unique_dict[source] = 0
            for sr, tg, vl in zip_lists:
                if sr == source:
                    unique_dict[source] += vl
                    
    # считаем проценты
    for source, target, value in zip_lists:
        new_list.append(round(100 * value / unique_dict[source], 1))
    
    return new_list
        

In [131]:
def lists_for_plot(df_comp, source_indexes, colors, frac=10):
    
    """
    Создаем необходимые для отрисовки диаграммы переменные списков и возвращаем
    их в виде словаря
    
    Args:
        source_indexes (dict): словарь с именами и индексами source.
        colors (dict): словарь с цветами source.
        frac (int): ограничение на минимальный "объем" между узлами.
        
    Returns:
        dict: словарь со списками, необходимыми для диаграммы.
    """
    
    sources = []
    targets = []
    values = []
    labels = []
    link_color = []
    link_text = []
    
    # проходим по каждому шагу
    for step in tqdm(sorted(df_comp['step'].unique()), desc='Шаг'):
        # номер шага больше, чем мы взяли, то пропускаем итерацию
        if step + 1 not in source_indexes:
            continue
        # получаем индекс источника
        temp_dict_source = source_indexes[step]['sources_dict']

        # получаем индексы цели
        temp_dict_target = source_indexes[step+1]['sources_dict']
        
        # проходим по каждой возможной паре, считаем количество таких пар
        for source, index_source in tqdm(temp_dict_source.items()):
            for target, index_target in temp_dict_target.items():
                # делаем срез данных и считаем количество id            
                temp_df = df_comp[(df_comp['step'] == step)&(df_comp['source'] == source)&(df_comp['target'] == target)]
                value = len(temp_df)
                # проверяем минимальный объем потока и добавляем нужные данные
                if value > frac:
                    sources.append(index_source)
                    targets.append(index_target)
                    values.append(value)
                    # делаем поток прозрачным для лучшего отображения
                    link_color.append(colors[source].replace(', 1)', ', 0.2)'))
    labels = []
    colors_labels = []
    for step in source_indexes:
        for name in source_indexes[step]['sources']:
            labels.append(name)
            colors_labels.append(colors[name])
            
    # посчитаем проценты всех потоков
    perc_values = percent_users(sources, targets, values)
    
    # добавим значения процентов для howertext
    link_text = []
    for perc in perc_values:
        link_text.append(f"{perc}%")
    
    # возвратим словарь с вложенными списками
    return {'sources': sources, 
            'targets': targets, 
            'values': values, 
            'labels': labels, 
            'colors_labels': colors_labels, 
            'link_color': link_color, 
            'link_text': link_text}
        

In [142]:
# создаем словарь
data_for_plot = lists_for_plot(df_comp, source_indexes, colors_dict)

Шаг:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

Теперь создадим функцию, которая создаст объект plotly (сам график)

In [145]:
def plot_senkey_diagram(data_dict):    
    
    """
    Функция для генерации объекта диаграммы Сенкей 
    
    Args:
        data_dict (dict): словарь со списками данных для построения.
        
    Returns:
        plotly.graph_objs._figure.Figure: объект изображения.
    """
    
    fig = go.Figure(data=[go.Sankey(
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
          pad = 50,
          thickness = 15,
          line = dict(color = "black", width = 0.1),
          label = data_dict['labels'],
          color = data_dict['colors_labels']
        ),
        link = dict(
          source = data_dict['sources'],
          target = data_dict['targets'],
          value = data_dict['values'],
          label = data_dict['link_text'],
          color = data_dict['link_color']
      ))])
    fig.update_layout(title_text="Sankey Diagram", font_size=10, width=3000, height=1200)
    
    # возвращаем объект диаграммы
    return fig
  

In [146]:

# сохраняем диаграмму в переменную
senkey_diagram = plot_senkey_diagram(data_for_plot)

In [147]:
senkey_diagram